In [7]:
import argparse
import torch
import time

from quantization import quantize

from SwissArmyTransformer import get_args, get_tokenizer
from SwissArmyTransformer.arguments import initialize_distributed
from SwissArmyTransformer.training import load_checkpoint
from SwissArmyTransformer.model import GLM130B, GLMModel
from SwissArmyTransformer.model.mixins import CachedAutoregressiveMixin
from SwissArmyTransformer.tokenization.glm import GPT2BPETokenizer
from SwissArmyTransformer.tokenization.icetk_glm_130B import _IceTokenizer



Please install apex to use fused_layer_norm, fall back to torch.nn.LayerNorm
Please install apex to use FusedScaleMaskSoftmax, otherwise the inference efficiency will be greatly reduced


In [10]:
# get_tokenizer(args = None, tokenizer_type = 'glm_GPT2BPETokenizer')
gpt2bpeTokenizer = GPT2BPETokenizer(model_type_or_path='gpt2', add_block_symbols=True, add_task_mask=True, add_decoder_mask=True)

In [18]:
reverse_vocab = {v: k for k, v in gpt2bpeTokenizer.vocab.items()}

In [8]:
icetkTokenizer = _IceTokenizer()

In [17]:
path = '/home/supercomputer.ai/GLM/GLM-130B/evaluation/evaluation_0804.tar/evaluation_0804/evaluation/lambada/lambada/please_next_word/gen/'
original_path = '/home/supercomputer.ai/GLM/GLM-130B/evaluation/evaluation_0804.tar/evaluation_0804/evaluation/lambada_original/'
# load test.jsonl   
import json
with open(path + 'test.jsonl') as f:
    test = [json.loads(line) for line in f]
# load validation.jsonl
with open(path + 'validation.jsonl') as f:
    validation = [json.loads(line) for line in f]

# load test_original.jsonl
with open(original_path + 'test.jsonl') as f:
    test_original = [json.loads(line) for line in f]
# load validation_original.jsonl
with open(original_path + 'validation.jsonl') as f:
    validation_original = [json.loads(line) for line in f]

In [29]:
icetkTokenizer.tokenize(test[0]['inputs_pretokenized']) == test[0]['inputs']

True

In [30]:
icetkTokenizer.tokenize(test[0]['targets_pretokenized'][0]) == test[0]['targets'][0]

True

In [55]:
# test[0]['targets_pretokenized']

gpt2bpeTokenizer._encode(test[0]['targets_pretokenized'][0])

[12683, 82]

In [20]:
test[0]['inputs_pretokenized']

"in my palm is a clear stone , and inside it is a small ivory statuette . a guardian angel . `` figured if you 're going to be out at night getting hit by cars , you might as well have some backup . '' i look at him , feeling stunned . like this is some sort of sign . but as i stare at harlin , his mouth curved in a confident grin , i do n't care about"

In [28]:
for i in range(len(test)):
    print(i)
    print(20005 in test[1]['inputs'])

0
False
1
False
2
False
3
False
4
False


In [29]:
test[0]

{'inputs_pretokenized': "in my palm is a clear stone , and inside it is a small ivory statuette . a guardian angel . `` figured if you 're going to be out at night getting hit by cars , you might as well have some backup . '' i look at him , feeling stunned . like this is some sort of sign . but as i stare at harlin , his mouth curved in a confident grin , i do n't care about",
 'inputs': [259,
  616,
  18057,
  318,
  257,
  1598,
  7815,
  837,
  290,
  2641,
  340,
  318,
  257,
  1402,
  32630,
  1185,
  84,
  5857,
  764,
  257,
  21688,
  18304,
  764,
  7559,
  11638,
  611,
  345,
  705,
  260,
  1016,
  284,
  307,
  503,
  379,
  1755,
  1972,
  2277,
  416,
  5006,
  837,
  345,
  1244,
  355,
  880,
  423,
  617,
  11559,
  764,
  10148,
  1312,
  804,
  379,
  683,
  837,
  4203,
  19987,
  764,
  588,
  428,
  318,
  617,
  3297,
  286,
  1051,
  764,
  475,
  355,
  1312,
  24170,
  379,
  3971,
  2815,
  837,
  465,
  5422,
  26929,
  287,
  257,
  6563,
  16770,
  837,

In [33]:
# decode test_original[0]['inputs'] one by one
for i in range(len(test_original[0]['inputs'])):
    print(i)
    print(test_original[0]['inputs'][i])
    print(icetkTokenizer.IdToToken(test_original[0]['inputs'][i]))
    print('-----')


0
20105
in
-----
1
20162
my
-----
2
35976
palm
-----
3
20107
is
-----
4
20104
a
-----
5
20902
clear
-----
6
25492
stone
-----
7
20005

-----
8
20006
,
-----
9
20102
and
-----
10
20916
inside
-----
11
20112
it
-----
12
20107
is
-----
13
20104
a
-----
14
20445
small
-----
15
66899
ivory
-----
16
35008
statue
-----
17
30510
tte
-----
18
20005

-----
19
20007
.
-----
20
20104
a
-----
21
42414
guardian
-----
22
41440
angel
-----
23
20005

-----
24
20007
.
-----
25
146337
``
-----
26
28953
figured
-----
27
20141
if
-----
28
20120
you
-----
29
20068
'
-----
30
20203
re
-----
31
20379
going
-----
32
20103
to
-----
33
20113
be
-----
34
20165
out
-----
35
20124
at
-----
36
20961
night
-----
37
20760
getting
-----
38
21298
hit
-----
39
20117
by
-----
40
23108
cars
-----
41
20005

-----
42
20006
,
-----
43
20120
you
-----
44
20425
might
-----
45
20114
as
-----
46
20233
well
-----
47
20127
have
-----
48
20188
some
-----
49
26960
backup
-----
50
20005

-----
51
20007
.
-----
52
146010
''
-----
53
20

In [71]:
for i in range(len(test_original)):
    # check if target has length 1
    if len(test_original[i]['inputs']) != 1:
        print(test_original[i]['inputs'])

[20105, 20162, 35976, 20107, 20104, 20902, 25492, 20005, 20006, 20102, 20916, 20112, 20107, 20104, 20445, 66899, 35008, 30510, 20005, 20007, 20104, 42414, 41440, 20005, 20007, 146337, 28953, 20141, 20120, 20068, 20203, 20379, 20103, 20113, 20165, 20124, 20961, 20760, 21298, 20117, 23108, 20005, 20006, 20120, 20425, 20114, 20233, 20127, 20188, 26960, 20005, 20007, 146010, 20227, 20444, 20124, 20302, 20005, 20006, 22996, 49613, 20005, 20007, 20176, 20121, 20107, 20188, 21911, 20101, 21704, 20005, 20007, 20137, 20114, 20227, 51985, 20124, 25896, 22386, 20005, 20006, 20147, 24425, 35254, 20105, 20104, 27244, 20005, 45542, 20005, 20006, 20227, 20161, 20461, 20022, 20119, 20568, 20158]
[20513, 20221, 20104, 23320, 20103, 20392, 20102, 20227, 20068, 20338, 21571, 20120, 20124, 20100, 35879, 20106, 20005, 20007, 146010, 20252, 20068, 20138, 22942, 20280, 21217, 20244, 20214, 25024, 20005, 20007, 146337, 20184, 20236, 20103, 20392, 20005, 20007, 20140, 32813, 20461, 20022, 20119, 20113, 20109, 

In [58]:
gpt2bpeTokenizer.text_tokenizer.vocab['sign']

12683

In [49]:
gpt2bpeTokenizer.tokenize(test[0]['targets_pretokenized'][0])

[12683, 82]

In [16]:
gpt2bpeTokenizer



<bound method Tokenizer.IdToToken of <SwissArmyTransformer.tokenization.glm.tokenization.GPT2BPETokenizer object at 0x7faf2d06e8b0>>

In [20]:
sample = [50259,   259,   616, 18057,   318,   257,  1598,  7815,   837,   290,
          2641,   340,   318,   257,  1402, 32630,  1185,    84,  5857,   764,
           257, 21688, 18304,   764,  7559, 11638,   611,   345,   705,   260,
          1016,   284,   307,   503,   379,  1755,  1972,  2277,   416,  5006,
           837,   345,  1244,   355,   880,   423,   617, 11559,   764, 10148,
          1312,   804,   379,   683,   837,  4203, 19987,   764,   588,   428,
           318,   617,  3297,   286,  1051,   764,   475,   355,  1312, 24170,
           379,  3971,  2815,   837,   465,  5422, 26929,   287,   257,  6563,
         16770,   837,  1312,   466,   299,   470,  1337,   546, 50263, 50256,
         50257,     0,     0,     0,     0,     0]

# decode it using gpt2bpeTokenizer.vocab
for i in sample:
    print(reverse_vocab[i])

[CLS]
in
Ġmy
Ġpalm
Ġis
Ġa
Ġclear
Ġstone
Ġ,
Ġand
Ġinside
Ġit
Ġis
Ġa
Ġsmall
Ġivory
Ġstat
u
ette
Ġ.
Ġa
Ġguardian
Ġangel
Ġ.
Ġ``
Ġfigured
Ġif
Ġyou
Ġ'
re
Ġgoing
Ġto
Ġbe
Ġout
Ġat
Ġnight
Ġgetting
Ġhit
Ġby
Ġcars
Ġ,
Ġyou
Ġmight
Ġas
Ġwell
Ġhave
Ġsome
Ġbackup
Ġ.
Ġ''
Ġi
Ġlook
Ġat
Ġhim
Ġ,
Ġfeeling
Ġstunned
Ġ.
Ġlike
Ġthis
Ġis
Ġsome
Ġsort
Ġof
Ġsign
Ġ.
Ġbut
Ġas
Ġi
Ġstare
Ġat
Ġhar
lin
Ġ,
Ġhis
Ġmouth
Ġcurved
Ġin
Ġa
Ġconfident
Ġgrin
Ġ,
Ġi
Ġdo
Ġn
't
Ġcare
Ġabout
[gMASK]
<|endoftext|>
<|startofpiece|>
!
!
!
!
!


In [ ]:
import copy
test_gpt2bpeTokenizer = copy.deepcopy(test)
for i in range(len(test_gpt2bpeTokenizer)):
    test_gpt2bpeTokenizer[i]['inputs'] = gpt2bpeTokenizer.tokenize(test_gpt2bpeTokenizer[i]['inputs_pretokenized'])
    # check if test_gpt2bpeTokenizer[i]['targets'] is length 1
    if not len(test_gpt2bpeTokenizer[i]['targets_pretokenized']) == 1:
        print('error')
    test_gpt2bpeTokenizer[i]['targets'][0] = \
        gpt2bpeTokenizer.tokenize(test_gpt2bpeTokenizer[i]['targets_pretokenized'][0])
    # check if test_gpt2bpeTokenizer[i]['targets'][0] is length 1
    if not len(test_gpt2bpeTokenizer[i]['targets']) == 1:
        print('error')

# save it back to test.jsonl
with open(path + 'test_gpt2bpeTokenizer.jsonl', 'w') as f:
    for i in range(len(test_gpt2bpeTokenizer)):
        f.write(json.dumps(test_gpt2bpeTokenizer[i]))
        f.write('\n')


# do the same for validation
validation_gpt2bpeTokenizer = copy.deepcopy(validation)
for i in range(len(validation_gpt2bpeTokenizer)):
    validation_gpt2bpeTokenizer[i]['inputs'] = gpt2bpeTokenizer.tokenize(validation_gpt2bpeTokenizer[i]['inputs_pretokenized'])
    # check if validation_gpt2bpeTokenizer[i]['targets'] is length 1
    if not len(validation_gpt2bpeTokenizer[i]['targets_pretokenized']) == 1:
        print('error')
    validation_gpt2bpeTokenizer[i]['targets'][0] = \
        gpt2bpeTokenizer.tokenize(validation_gpt2bpeTokenizer[i]['targets_pretokenized'][0])
    # check if validation_gpt2bpeTokenizer[i]['targets'][0] is length 1
    if not len(validation_gpt2bpeTokenizer[i]['targets']) == 1:
        print('error')

# save it back to validation.jsonl
with open(path + 'validation_gpt2bpeTokenizer.jsonl', 'w') as f:
    for i in range(len(validation_gpt2bpeTokenizer)):
        f.write(json.dumps(validation_gpt2bpeTokenizer[i]))
        f.write('\n')

In [38]:
list_of_ids = [ 20105,  20162,  35976,  20107,  20104,  20902,  25492,  20005,  20006,
          20102,  20916,  20112,  20107,  20104,  20445,  66899,  35008,  30510,
          20005,  20007,  20104,  42414,  41440,  20005,  20007, 146337,  28953,
          20141,  20120,  20068,  20203,  20379,  20103,  20113,  20165,  20124,
          20961,  20760,  21298,  20117,  23108,  20005,  20006,  20120,  20425,
          20114,  20233,  20127,  20188,  26960,  20005,  20007, 146010,  20227,
          20444,  20124,  20302,  20005,  20006,  22996,  49613,  20005,  20007,
          20176,  20121,  20107,  20188,  21911,  20101,  21704,  20005,  20007,
          20137,  20114,  20227,  51985,  20124,  25896,  22386,  20005,  20006,
          20147,  24425,  35254,  20105,  20104,  27244,  20005,  45542,  20005,
          20006,  20227,  20161,  20461,  20022,  20119,  20568,  20158, 150001,
         150004,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0]

In [45]:
list_of_ids =[146337,  20488,  20111,  20005,  20006,  23094,  22511,  20005,
           20006, 146010,  20005,  47042,  32674,  20166,  20005,  20006,
           42258,  20181,  20175,  20100,  20103,  20186,  20005,  20006,
           20220,  34819,  20103,  26827,  20214,  22748,  20005,  20007,
          146337,  20643,  20221,  20392,  20142,  57360,  20102,  20541,
           20120,  20360,  20108,  20104,  21499,  20101,  20898,  20005,
           20007, 146010, 146337,  20252,  22403,  20106,  20109,  20322,
           20005,  20031, 146010, 146337,  21031,  20406,  24568,  20005,
           20007,  20252,  20331,  21229,  20104,  21774,  40794,  20005,
           20006,  20428,  20005,  20006,  20137,  20252,  20068,  20338,
           20165,  52416,  20280,  21221,  21443,  20005,  20007, 146010,
           20579,  29450,  22374,  20252,  20171,  20104,  20538,  20103,
           21583,  20158, 150001, 150004,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
           30034,  20005,  20006,  20137,  20252,  20116,  21221,  20547,
           20109,  20005,  47042,  32674,  20116,  20312,  20005,  20007,
           20005,  47042,  26254,  20116,  20104,  20315,  21830,  20005,
           20006,  20102,  20252,  20116,  20104,  20451,  21913,  20005,
           20007,  20579,  29450,  20116,  27723,  20252,  20171,  20286,
           20214,  20005,  20007,  20252,  20116,  27723,  20109,  20252,
           20171,  20488,  20103,  20100,  21012,  20005,  20007,  20112,
           20116,  20104,  20538,  20101,  21450,  20005]

In [2]:
list_of_ids = [20100,  21704,  20005,  20007,  20227]

In [46]:
for index in list_of_ids:
    print(icetkTokenizer.IdToToken(index))

``
come
on

,
baby
girl

,
''

mary
jo
said

,
scoop
ing
up
the
to
y

,
then
bending
to
retrieve
her
daughter

.
``
let
me
change
your
diaper
and
put
you
down
for
a
couple
of
hours

.
''
``
she
sleep
s
that
long

?
''
``
almost
every
afternoon

.
she
still
takes
a
morning
nap

,
too

,
but
she
'
ll
out
grow
those
pretty
soon

.
''
l
ori
knew
she
had
a
lot
to
learn
about
[[gMASK]]
[sop]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
[pad]
babies

,
but
she
was
pretty
sure
that

mary
jo
was
right

.

mary
jo
was
a
good
mother

,
and
she
was
a
great
friend

.
l
ori
was
glad
she
had
found
her

.
she
was
glad
that
she
had
come
to
the
party

.
it
was
a
lot
of
fun



In [8]:
# [[[34504, 20144, 20022, 20106, 20156],
#   [34504, 20005, 20007, 20141, 20144],
#   [34504, 20005, 20007, 20153, 20270],
#   [34504, 20005, 20007, 20144, 23203],
#   [34504, 20005, 20007, 20112, 20530],
#   [34504, 20005, 20007, 20141, 20120],
#   [34504, 20005, 20007, 20102, 20141],
#   [34504, 20005, 20007, 20121, 20107],
#   [34504, 20005, 20007, 20173, 20118],
#   [34504, 20005, 20007, 20120, 20129],
#   [34504, 20005, 20007, 20102, 20144],
#   [34504, 20005, 20007, 20120, 20270],
#   [34504, 20005, 20007, 20153, 20127],
#   [34504, 20005, 20007, 20153, 20118],
#   [34504, 20005, 20007, 20144, 20132],
#   [34504, 20005, 20007, 20109, 20530]]]

icetkTokenizer.DecodeIds([34504, 20144, 20022, 20106, 20156])

AttributeError: '_IceTokenizer' object has no attribute 'DecodeIds'

In [5]:
options = [20202, 20101, 20109, 20005, 20007]
for option in options:
    print(icetkTokenizer.IdToToken(option))

any
of
that

.


In [9]:
options = [[20202, 20101, 20109, 20005, 20007],
            [20100, 21704, 20005, 20007, 20227],
            [20202, 20101, 20112, 20005, 20007],
            [20202, 23261, 20005, 20007, 20227],
            [20196, 20112, 20530, 20005, 20007],
            [20950, 20560, 20005, 20007, 20227],
            [20100, 41440, 20005, 20007, 20227],
            [23261, 25703, 20005, 20007, 20227],
            [20202, 21704, 20005, 20007, 20227],
            [23261, 20005, 20007, 20227, 20205],
            [20202, 20101, 20109, 25703, 20005],
            [20146, 20109, 20005, 20007, 20227],
            [20100, 23261, 20005, 20007, 20227],
            [20109, 20005, 20007, 20227, 20205],
            [23261, 20005, 20007, 20227, 20270],
            [20202, 20101, 20109, 20005, 20006]]

for option in options:
    for index in option:
        print(icetkTokenizer.IdToToken(index))
    print('--------------')

any
of
that

.
--------------
the
sign

.
i
--------------
any
of
it

.
--------------
any
signs

.
i
--------------
what
it
means

.
--------------
anything
else

.
i
--------------
the
angel

.
i
--------------
signs
anymore

.
i
--------------
any
sign

.
i
--------------
signs

.
i
just
--------------
any
of
that
anymore

--------------
all
that

.
i
--------------
the
signs

.
i
--------------
that

.
i
just
--------------
signs

.
i
know
--------------
any
of
that

,
--------------


In [10]:
icetkTokenizer.vocab()

AssertionError: 

In [44]:
icetkTokenizer.IdToToken(150005)

'[eop]'

In [12]:
icetkTokenizer.tokenizer._text_tokenizer

AttributeError: 'TextTokenizer' object has no attribute '_token_dict'

In [6]:
icetkTokenizer.IdToToken(0)

NameError: name 'icetkTokenizer' is not defined

In [2]:
# intermediate_variables/2023-01-24-15-36-10_run/h/predict_single_batch/outputs_batch/0.pkl
import pickle
# load the file
with open('/home/supercomputer.ai/intermediate_variables/2023-01-24-15-36-10_run/h/predict_single_batch/outputs_batch/0.pkl', 'rb') as f:
    data = pickle.load(f)

/home/supercomputer.ai/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# /home/supercomputer.ai/intermediate_variables/2023-01-25-08-13-48_run/home||supercomputer.ai||GLM-130B-small-branch2||GLM-130B||tasks||lambada||task.py/predict_single_batch/outputs_batch/0.pkl

import pickle
# load the file

with open('/home/supercomputer.ai/intermediate_variables/2023-01-25-08-13-48_run/home||supercomputer.ai||GLM-130B-small-branch2||GLM-130B||tasks||lambada||task.py/predict_single_batch/outputs_batch/0.pkl', 'rb') as f:
    data = pickle.load(f)

/home/supercomputer.ai/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pickle
output_targets_path = '/home/supercomputer.ai/glm-tom-git/2023-01-25-14-52-08_run/work||09127||tomyoung||ls6||glm||GLM-130B||evaluation||model.py/generate_text/output_targets/0.pkl'
with open(output_targets_path, 'rb') as f:
    output_targets = pickle.load(f)

/home/supercomputer.ai/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
output_targets_list = output_targets[0][0].tolist()

In [14]:
# use icetkTokenizer.IdToToken to get the tokens
list_of_tokens = []
for index in output_targets_list:
    list_of_tokens.append(icetkTokenizer.IdToToken(index))

In [38]:
# /home/supercomputer.ai/glm-tom-git/2023-01-25-14-52-08_run/work||09127||tomyoung||ls6||glm||GLM-130B||evaluation||model.py/batch_filling_sequence/tokens generated so far/0.pkl

import pickle
tokens_generated_so_far_path = '/home/supercomputer.ai/glm-tom-git/2023-01-25-14-52-08_run/work||09127||tomyoung||ls6||glm||GLM-130B||evaluation||model.py/'+\
    'batch_filling_sequence/tokens generated so far/99.pkl'
with open(tokens_generated_so_far_path, 'rb') as f:
    tokens_generated_so_far = pickle.load(f)

tokens_generated_so_far_list = tokens_generated_so_far[0][0].tolist()
for index in [tokens_generated_so_far_list[-1]]:
    print(icetkTokenizer.IdToToken(index))

.


In [37]:
tokens_generated_so_far_list[-1]

20007

In [24]:
output_targets_path = '/home/supercomputer.ai/glm-tom-git/2023-01-25-14-52-08_run/work||09127||tomyoung||ls6||glm||GLM-130B||evaluation||model.py/' + \
                    'generate_text/output_targets/0.pkl'
with open(output_targets_path, 'rb') as f:
    output_targets = pickle.load(f)

output_targets_list = output_targets[0][0].tolist()
for index in output_targets_list:
    print(icetkTokenizer.IdToToken(index))

the
sign

.
i
just
want
to
know
what
he
knows

.
so
i
ask
him

.

.
and
he
tells
me

.
he
tells
about
the
night
he
was
attacked

.
about
the
man
who
came
to
his
door

,
and
the
man
he
killed

.
the
man
with
the
angel

.
har
lin
tells
me
about
the
angel
he
found
in
the
alley

,
the
angel
that
was
carved
from
the
same
piece
of
ivory
as
the
guardian
angel

,
that
was
in
my
hand

.


In [26]:
len(output_targets_list)

100